In [ ]:
%%time

! pip install huggify-data

In [1]:
from datasets import load_dataset

ds = load_dataset("Aiwensile2/StorySeed")

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/4376 [00:00<?, ? examples/s]

In [2]:
ds

DatasetDict({
    train: Dataset({
        features: ['prompt', 'response'],
        num_rows: 4376
    })
})

In [3]:
ds['train']['prompt'][0]

'Please help me create a picture book with the theme of\nA curious kitten embarks on a whimsical journey through a magical forest, encountering friendly creatures along the way.\nThe format should include a title, followed by a cover page with an illustration only, and the remaining pages should each have both text and illustrations.'

In [4]:
ds['train']['response'][0]

'### Title: Whisker\'s Magical Forest Adventure\n\n---\n\n### Cover Page Illustration:\nA small, fluffy, gray kitten with wide, curious eyes standing at the edge of a lush, vibrant forest filled with colorful flowers and twinkling lights.\n\n---\n\n### Page 1 Illustration:\nWhiskers peeking out from behind a large oak tree, gazing at the path ahead.\n\n#### Text:\nIn the heart of a sun-dappled garden, little Whiskers the kitten found a hidden path leading into a mysterious forest. With a twitch of his nose and a flick of his tail, he decided it was the perfect day for an adventure.\n\n---\n\n### Page 2 Illustration:\nWhiskers meeting a cheerful squirrel with a bushy tail, sitting on a tree branch.\n\n#### Text:\nNot far into the forest, Whiskers met Squeaky the squirrel. "Where are you off to, little kitten?" Squeaky chirped. "I\'m exploring this magical forest," Whiskers replied with a purr. Squeaky nodded, "Follow me! I\'ll show you the way to the Whispering Willows."\n\n---\n\n### P

In [5]:
from tqdm import tqdm

```python
### Human: What does YSA stand for? ### Assistant: YSA stands for Youth Spirit Artworks.
```

In [6]:
questions = []
answers = []
texts = []

for i in tqdm(range(100)):
    ques = ds['train']['prompt'][i]
    answ = ' '.join(ds['train']['response'][i].split('### '))
    text = "### Human: " + ques + " ### Assistant: " + answ

    questions.append(ques)
    answers.append(answ)
    texts.append(text)

100%|██████████| 100/100 [00:01<00:00, 51.24it/s]


In [7]:
import pandas as pd

In [8]:
df = pd.DataFrame({'questions': questions, 'answers': answers, 'text': texts})
df.head()

,questions,answers,text
0,Please help me create a picture book with the ...,Title: Whisker's Magical Forest Adventure\n\n...,### Human: Please help me create a picture boo...
1,Please help me create a picture book with the ...,Title: **Mia and Mico’s Treasure Quest**\n\n ...,### Human: Please help me create a picture boo...
2,Please help me create a picture book with the ...,Title: The Enchanted Doorway\n\n---\n\n# Cove...,### Human: Please help me create a picture boo...
3,Please help me create a picture book with the ...,"Title: ""The Cave of Dragon Friends""\n\n---\n\...",### Human: Please help me create a picture boo...
4,Please help me create a picture book with the ...,Title: Pip's Quest for the Golden Cheese\n\n ...,### Human: Please help me create a picture boo...


In [9]:
df.to_csv('data.csv', index=False)

In [10]:
from google.colab import userdata

In [11]:
from huggify_data.push_modules import DataFrameUploader

# Example usage:
df = pd.read_csv('/content/data.csv')
uploader = DataFrameUploader(
    df,
    hf_token=userdata.get('HF_TOKEN'),
    repo_name='storySeed-v2', # this is whatever you want
    username='eagle0504' # your own user name
)
uploader.process_data()
uploader.push_to_hub()

Dataframe verified: columns 'questions' and 'answers' are present.
Data processed into DatasetDict.
Repository created: eagle0504/storySeed-v2


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset pushed to Hugging Face Hub.


In [12]:
from huggify_data.train_modules import *

# Parameters
model_name = "NousResearch/Llama-2-7b-chat-hf" # Recommended base model by Facebook (don't touch it)
dataset_name = "eagle0504/storySeed-v2" # Desired name, e.g., <hf_user_id>/<desired_name>
new_model = "storyseed-v2-llama2-model" # Desired name
huggingface_token = userdata.get('HF_TOKEN')

In [17]:
# Empty VRAM
import gc
gc.collect()

38

In [22]:
!nvidia-smi -q -d Memory


==============NVSMI LOG==============

Timestamp                                 : Thu Jul  4 16:51:54 2024
Driver Version                            : 535.104.05
CUDA Version                              : 12.2

Attached GPUs                             : 1
GPU 00000000:00:04.0
    FB Memory Usage
        Total                             : 15360 MiB
        Reserved                          : 257 MiB
        Used                              : 15101 MiB
        Free                              : 1 MiB
    BAR1 Memory Usage
        Total                             : 256 MiB
        Used                              : 5 MiB
        Free                              : 251 MiB
    Conf Compute Protected Memory Usage
        Total                             : 0 MiB
        Used                              : 0 MiB
        Free                              : 0 MiB



In [18]:
%%time

# Initiate
trainer = LlamaTrainer(model_name, dataset_name, new_model, huggingface_token)
peft_config = trainer.configure_lora()
training_args = trainer.configure_training_arguments(num_train_epochs=2)

# Training
trainer.train_model(training_args, peft_config)

Starting model loading...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/IPython/core/magics/execution.py:1335 in time            │
│                                                                                                  │
│   1332 │   │   else:                                                                             │
│   1333 │   │   │   st = clock2()                                                                 │
│   1334 │   │   │   try:                                                                          │
│ ❱ 1335 │   │   │   │   exec(code, glob, local_ns)                                                │
│   1336 │   │   │   │   out=None                                                                  │
│   1337 │   │   │   │   # multi-line %%time case                                                  │
│   1338 │   │   │   │   if expr_val is not None:                                                  │
│ in <module>:2                                                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggify_data/train_modules.py:26 in __init__             │
│                                                                                                  │
│    23 │   │   self.device_map = {"": 0}                                                          │
│    24 │   │   self.bnb_config = self.configure_bitsandbytes()                                    │
│    25 │   │   self.tokenizer = self.load_tokenizer()                                             │
│ ❱  26 │   │   self.model = self.load_model()                                                     │
│    27 │                                                                                          │
│    28 │   def configure_bitsandbytes(self):                                                      │
│    29 │   │   compute_dtype = torch.float16                                                      │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggify_data/train_modules.py:40 in load_model           │
│                                                                                                  │
│    37 │                                                                                          │
│    38 │   def load_model(self):                                                                  │
│    39 │   │   print("Starting model loading...")                                                 │
│ ❱  40 │   │   model = AutoModelForCausalLM.from_pretrained(                                      │
│    41 │   │   │   self.model_name,                                                               │
│    42 │   │   │   quantization_config=self.bnb_config,                                           │
│    43 │   │   │   device_map=self.device_map,                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:493 in          │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   490 │   │   │   )                                                                              │
│   491 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│   492 │   │   │   model_class = _get_model_class(config, cls._model_mapping)                     │
│ ❱ 493 │   │   │   return model_class.from_pretrained(                                            │
│   494 │   │   │   │   pretrained_model_name_or_path, *model

In [15]:
# Train and save | Run this in a new cell
trainer.merge_and_save_model()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'trainer' is not defined